In [9]:
import re
import json
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from features.features_encoder import FeaturesEncoder
from log_handler import LogHandlerManager

In [ ]:
R_STATE = 42
# Charger le fichier de configuration
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

In [ ]:
# Charger les données prétraitées
PATH_TO_CSV = '../../data/processed/merged_data_2019_2022.csv'
df = pd.read_csv(PATH_TO_CSV, low_memory=False)

log_handler_manager = LogHandlerManager(config["logs"])
log_handler_manager.start_listener()
logger = logging.getLogger(__name__)
logger.addHandler(log_handler_manager.get_queue_handler())
logger.setLevel(logging.DEBUG)

In [ ]:
# On découpe
skf = StratifiedKFold(n_splits= 5, shuffle= True, random_state= R_STATE)

In [ ]:
reference_size = len(df[df["grav"] == 3])

for fold, (train_index, test_index) in enumerate(skf.split(X, X['grav'])):
    print(f'Fold {fold +1 }')

    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = df["grav"].iloc[train_index], df["grav"].iloc[test_index]

    balanced_data = pd.DataFrame()
    encoder = FeaturesEncoder(config, logger)

    # Pour chaque modalité
    for class_label in X_train["grav"].unique():
        class_data = X_train[X_train["grav"] == class_label]

        if len(class_data) > reference_size:
            # Sous échantilloner
            class_data = resample(class_data, replace= False, n_samples= reference_size, random_state= R_STATE)
        else:
            class_data = resample(class_data, replace= True, n_samples= reference_size, random_state= R_STATE)
